Getting population data aggregated on municipali

In [58]:
import requests
import pandas as pd

# Retrieves population data from all municipalities

## Defining calls to Statistic Denmark API
# 2008 to date
FOLK1A = 'https://api.statbank.dk/v1/data/FOLK1A/CSV?valuePresentation=CodeAndValue&OMR%C3%85DE=*&Tid=*'

# 2005 to 2009
BEF1A07 = "https://api.statbank.dk/v1/data/BEF1A07/CSV?valuePresentation=CodeAndValue&OMR%C3%85DE=*&Tid=*"

# 1979 to 2006
BEF1A = "https://api.statbank.dk/v1/data/BEF1A/CSV?valuePresentation=CodeAndValue&OMR%C3%85DE=*&Tid=*"

# Retrieving data
FOLK1A_data = pd.read_csv(FOLK1A, sep = ";")
BEF1A07_data = pd.read_csv(BEF1A07, sep = ";")
BEF1A_data = pd.read_csv(BEF1A, sep = ";")

In [59]:
# cleaning the data
# FOLK1A needs additional step

def clean_pop_data(df):
    df['muni_code'] = df['OMRÅDE'].str.extract(r'(\d{3})')   # extracting municipality code
    df['muni_name'] = df['OMRÅDE'].str.extract(r'(\d{3}\s+(.*))')[1] # extracting municipalit name
    df['year'] = df['TID'].str.split().str[0]
    df = df\
        .drop(columns=["OMRÅDE", 'TID'])\
        .rename(columns={'INDHOLD':'pop'})
    return df

FOLK1A_data = clean_pop_data(FOLK1A_data)
BEF1A_data = clean_pop_data(BEF1A_data)
BEF1A07_data = clean_pop_data(BEF1A07_data)

In [60]:
FOLK1A_data["muni_code"] = FOLK1A_data["muni_code"].astype("Int64")
BEF1A_data["muni_code"] = BEF1A_data["muni_code"].astype("Int64")
BEF1A07_data["muni_code"] = BEF1A07_data["muni_code"].astype("Int64")

In [61]:
# additional cleaning of FOLK1A data
FOLK1A_data = FOLK1A_data[FOLK1A_data['year'].str.endswith('1')]
FOLK1A_data['year'] = FOLK1A_data['year'].str.replace('K1', "")


concatenating the newish data

In [62]:
pop_data = pd.concat([FOLK1A_data, BEF1A07_data]).reset_index(drop = True).drop_duplicates()
nye = pop_data[["muni_code", 'year', "pop"]]

In [63]:
#now working on older data

# vi indlæser vores data på hvilke nye muni_codes de har
konkordans = pd.read_csv("nye_og_gamle_kommuner.csv")


konkordans['old_muni_code'] = konkordans['old_muni_code'].astype('Int64')
konkordans['muni_code'] = konkordans['muni_code'].astype('Int64')


BEF1A_data['muni_code'] = BEF1A_data['muni_code'].astype('Int64')
BEF1A_data = BEF1A_data.rename(columns={"muni_code":"old_muni_code"})

gamle = BEF1A_data\
    .merge(konkordans, how='left', on = 'old_muni_code')\
    .groupby(["muni_code", "year"])["pop"].agg("sum").reset_index()


In [64]:
konkordans['old_muni_code'] = konkordans['old_muni_code'].astype('Int64')
konkordans['muni_code'] = konkordans['muni_code'].astype('Int64')


,muni_code,year,pop
0,101,1979,505974
1,101,1980,498850
2,101,1981,493771
3,101,1982,490587
4,101,1983,486593
...,...,...,...
2739,860,2002,68325
2740,860,2003,68022
2741,860,2004,67898
2742,860,2005,67816


In [65]:
# Combining total
totalen = pd.concat([nye, gamle])
totalen["year"] = totalen["year"].astype("Int64")

In [66]:
# writing to file
totalen.to_csv("population_data.csv")
